<a href="https://colab.research.google.com/github/ciepielajan/GeoFabrik_Data_Extract/blob/main/GeoFabrik_raport_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import folium
import pandas as pd
from plotnine import *
!pip -q install geopandas
from geopandas import GeoDataFrame

%matplotlib inline

     |████████████████████████████████| 1.0MB 5.8MB/s 
     |████████████████████████████████| 14.8MB 269kB/s 
     |████████████████████████████████| 6.5MB 41.8MB/s 


In [2]:
link = "http://download.geofabrik.de/europe/poland/mazowieckie-latest-free.shp.zip"

In [3]:
!wget $link  -O map.shp.zip

--2021-04-06 09:08:52--  http://download.geofabrik.de/europe/poland/mazowieckie-latest-free.shp.zip
Resolving download.geofabrik.de (download.geofabrik.de)... 138.201.219.183, 116.202.112.212, 88.99.142.44
Connecting to download.geofabrik.de (download.geofabrik.de)|138.201.219.183|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 339381802 (324M) [application/zip]
Saving to: ‘map.shp.zip’

map.shp.zip         100%[===================>] 323.66M  20.6MB/s    in 19s     

2021-04-06 09:09:11 (17.1 MB/s) - ‘map.shp.zip’ saved [339381802/339381802]



In [4]:
!unzip -q map.shp.zip -d /content/maps

In [5]:
from os import listdir

files_dir =  listdir("/content/maps")
files_list = []
for names in files_dir:
    if names.endswith(".shp"):
        files_list.append(names)
        
files_list = sorted(files_list)

#podgląd
print(pd.Series(files_list))

0     gis_osm_buildings_a_free_1.shp
1       gis_osm_landuse_a_free_1.shp
2       gis_osm_natural_a_free_1.shp
3         gis_osm_natural_free_1.shp
4        gis_osm_places_a_free_1.shp
5          gis_osm_places_free_1.shp
6          gis_osm_pofw_a_free_1.shp
7            gis_osm_pofw_free_1.shp
8          gis_osm_pois_a_free_1.shp
9            gis_osm_pois_free_1.shp
10       gis_osm_railways_free_1.shp
11          gis_osm_roads_free_1.shp
12      gis_osm_traffic_a_free_1.shp
13        gis_osm_traffic_free_1.shp
14    gis_osm_transport_a_free_1.shp
15      gis_osm_transport_free_1.shp
16        gis_osm_water_a_free_1.shp
17      gis_osm_waterways_free_1.shp
dtype: object


In [6]:
%time
data_frames = {}
for name in files_list:
  name_df = name.split(".")[0]
  data_frames[name_df] = GeoDataFrame.from_file(f'maps/{name}', encoding='utf-8')

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 11.2 µs


In [7]:
#podgląd
data_frames.keys()

dict_keys(['gis_osm_buildings_a_free_1', 'gis_osm_landuse_a_free_1', 'gis_osm_natural_a_free_1', 'gis_osm_natural_free_1', 'gis_osm_places_a_free_1', 'gis_osm_places_free_1', 'gis_osm_pofw_a_free_1', 'gis_osm_pofw_free_1', 'gis_osm_pois_a_free_1', 'gis_osm_pois_free_1', 'gis_osm_railways_free_1', 'gis_osm_roads_free_1', 'gis_osm_traffic_a_free_1', 'gis_osm_traffic_free_1', 'gis_osm_transport_a_free_1', 'gis_osm_transport_free_1', 'gis_osm_water_a_free_1', 'gis_osm_waterways_free_1'])

In [8]:
region = " - ".join(link.split("http://download.geofabrik.de/")[1].split("-latest")[0].split("/")).upper()

raport = ""
raport += f"# Raport region: {region}"
raport += "\n\n"

for name in data_frames:
  title = name.split("gis_osm_")[1].split("_free_1")[0].upper()
  raport += f"## {title}"
  raport += "\n"
  raport += f"Nazwa plik: {name}.shp"
  raport += "\n\n"
  raport += f"Ilość rekordów: {data_frames.get(name).shape[0]} "
  raport += "\n\n"
  raport += "Sample index "
  raport += "\n"
  raport += data_frames.get(name).sample(1).to_markdown()
  raport += "\n\n"
  raport += "Value counts for fclass"
  raport += "\n"
  raport += data_frames.get(name)["fclass"].value_counts().to_markdown()
  raport += "\n\n"

raport

'# Raport region: EUROPE - POLAND - MAZOWIECKIE\n\n## BUILDINGS_A\nNazwa plik: gis_osm_buildings_a_free_1.shp\n\nIlość rekordów: 1881692 \n\nSample index \n|         |    osm_id |   code | fclass   | name   | type     | geometry                                                                                                                     |\n|--------:|----------:|-------:|:---------|:-------|:---------|:-----------------------------------------------------------------------------------------------------------------------------|\n| 1838277 | 921453246 |   1500 | building |        | bungalow | POLYGON ((20.3717179 51.9040274, 20.371729 51.9040646, 20.3718541 51.9040502, 20.3718429 51.9040131, 20.3717179 51.9040274)) |\n\nValue counts for fclass\n|          |      fclass |\n|:---------|------------:|\n| building | 1.88169e+06 |\n\n## LANDUSE_A\nNazwa plik: gis_osm_landuse_a_free_1.shp\n\nIlość rekordów: 362124 \n\nSample index \n|        |    osm_id |   code | fclass   | name   | geo

In [11]:
file_name= region.lower().replace(" ","")
file_name = f"raport-{file_name}.md"
f = open(file_name, "a")
f.write(raport)
f.close()

#auto-run download to local
from google.colab import files
files.download(file_name)